In [1]:
import mxnet as mx
from mxnet import autograd as ag
from mxnet import gluon
from mxnet import init
from mxnet import nd
from time import time
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## parameter

In [2]:
project_path = '/home/carsmart/users/xiaoming/Distracted_Driver_Detection/'
path_list = project_path + 'train_lst/train.lst'
path_rec = project_path + 'train_lst/train.rec'
val_path_list = project_path + 'train_lst/val_val.lst'
val_path_rec = project_path + 'train_lst/val_val.rec'
test_path_list = project_path + 'test_lst/test_test.lst'
test_path_rec = project_path + 'test_lst/test_test.rec'

save_path = project_path + 'model/'

(mean_r, mean_g, mean_b) = (0.485 * 255, 0.456 * 255, 0.406 * 255)
(std_r, std_g, std_b) = (0.229 * 225, 0.224 * 255, 0.225 * 255)

batch_size = 16
ctx = [mx.gpu(1)]

## iter

In [3]:
def get_iter(kv, resize, data_shape):
    train_iter = mx.io.ImageRecordIter(
        path_imglist = path_list, 
        path_imgrec = path_rec, 
        resize = resize, 
        data_shape = data_shape, 
        batch_size = batch_size, 
        rand_mirror = False, 
        rand_crop = False, 
        mean_r = mean_r, 
        mean_g = mean_g, 
        mean_b = mean_b, 
        std_r = std_r, 
        std_g = std_g, 
        std_b = std_b, 
        num_parts = kv.num_workers, 
        part_index = kv.rank, 
        shuffle = True
    )
    train_iter = mx.io.PrefetchingIter(train_iter)
    
    val_iter = mx.io.ImageRecordIter(
        path_imglist = val_path_list, 
        path_imgrec = val_path_rec, 
        resize = resize, 
        data_shape = data_shape, 
        batch_size = batch_size, 
        rand_mirror = False, 
        rand_crop = False, 
        mean_r = mean_r, 
        mean_g = mean_g, 
        mean_b = mean_b, 
        std_r = std_r, 
        std_g = std_g, 
        std_b = std_b, 
        num_parts = kv.num_workers, 
        part_index = kv.rank
    )
    
    test_iter = mx.io.ImageRecordIter(
        path_imglist = test_path_list, 
        path_imgrec = test_path_rec, 
        resize = resize, 
        data_shape = data_shape, 
        batch_size = batch_size, 
        rand_mirror = False, 
        rand_crop = False, 
        mean_r = mean_r, 
        mean_g = mean_g, 
        mean_b = mean_b, 
        std_r = std_r, 
        std_g = std_g, 
        std_b = std_b, 
        num_parts = kv.num_workers, 
        part_index = kv.rank
    )
    
    return(train_iter, val_iter, test_iter)

## model define
use model define in mxnet.gluon.model_zoo

In [8]:
pretrained_net = models.get_model(name = 'densenet161', pretrained = True, ctx = ctx, prefix = 'densenet161_')
net = models.get_model(name = 'densenet161', classes = 10, prefix = 'densenet161_')
net.features = pretrained_net.features
net.output.initialize(init.Xavier(), ctx = ctx)
net.hybridize()

## train define

In [5]:
def get_batch(batch, ctx):
    """return data and label on ctx"""
    if isinstance(batch, mx.io.DataBatch):
        data = batch.data[0]
        label = batch.label[0]
    else:
        data, lable = batch
    return (gluon.utils.split_and_load(data, ctx), 
           gluon.utils.split_and_load(label, ctx), 
           data.shape[0])

def evaluate_accuracy(data_iterator, net, ctx = [mx.cpu()]):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc = nd.array([0])
    losses = nd.array([0])
    n = 0.
    if isinstance(data_iterator, mx.io.MXDataIter):
        data_iterator.reset()
    for batch in data_iterator:
        data, label, batch_size = get_batch(batch, ctx)
        for x, y in zip(data, label):
            outputs = net(x)
            acc += nd.sum(outputs.argmax(axis = 1) == y).copyto(mx.cpu())
            losses += nd.sum(loss(outputs, y)).copyto(mx.cpu())
            n += y.size
        nd.waitall() # don't push too many operators into backend
    return acc.asscalar() / n, losses.asscalar() / n

In [6]:
def train(kv, net, loss, trainer, ctx, num_epochs, 
          print_batches = None, save_epochs = 10000):
    """train a net work"""
    print("Start training on ", ctx)
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    for epoch in range(num_epochs):
        (train_data, val_data, _) = get_iter(kv, resize = resize, data_shape = data_shape)
        train_loss, train_acc, n, m = 0.0, 0.0, 0.0, 0.0
        if isinstance(train_data, (mx.io.PrefetchingIter, mx.io.MXDataIter)):
            train_data.reset()
        start = time()
        for i, batch in enumerate(train_data):
            data, label, batch = get_batch(batch, ctx)
            losses = []
            with ag.record():
                outputs = [net(x) for x in data]
                losses = [loss(yhat, y) for yhat, y in zip(outputs, label)]
            for l in losses:
                l.backward()
            train_acc += sum([(yhat.argmax(axis = 1) == y).sum().asscalar() 
                             for yhat, y in zip(outputs, label)])
            train_loss += sum([l.sum().asscalar() for l in losses])
            trainer.step(batch_size)
            n += batch_size
            m += sum([y.size for y in label])
            if print_batches and (i + 1) % print_batches == 0:
                print("Data %d. Loss: %f, Train acc %f, Time %.1f sec" % (
                    n, train_loss / n, train_acc / m, time() - start
                ))
        test_acc, test_loss = evaluate_accuracy(val_data, net, ctx)
        # test_acc = 0
        # if not epoch % save_epochs:
        #     net.collect_params().save(save_path + 'method2' + str(epoch) + '00.params')
        print("Epoch %d. Loss: %.3f, Train acc %.2f, Test acc %.2f, Test loss %.3f, Time %.1f sec, learning_rate %f" %(
            epoch, train_loss / n, train_acc / m, test_acc, test_loss, time() - start, trainer.learning_rate
        ))

## train

In [9]:
kv = mx.kvstore.create("local")
loss = gluon.loss.SoftmaxCrossEntropyLoss()
learning_rate = 5e-5
wd = 1e-6
lr_step = 400
lr_factor = 0.9
resize = 224
data_shape = (3, 224, 224)

num_epochs = 10
lr_scheduler = mx.lr_scheduler.FactorScheduler(step = lr_step, factor = lr_factor)

trainer = gluon.Trainer(net.collect_params(), 
                       'adam', {'learning_rate': learning_rate})
train(kv, net, loss, trainer, ctx, num_epochs = num_epochs, 
      print_batches = 1000, save_epochs = 1)

Start training on  [gpu(1)]
Data 16000. Loss: 0.148729, Train acc 0.962625, Time 276.3 sec
Epoch 0. Loss: 0.135, Train acc 0.97, Test acc 0.81, Test loss 0.602, Time 333.5 sec, learning_rate 0.000050
Data 16000. Loss: 0.007701, Train acc 0.998875, Time 277.2 sec
Epoch 1. Loss: 0.007, Train acc 1.00, Test acc 0.82, Test loss 0.525, Time 334.0 sec, learning_rate 0.000050
Data 16000. Loss: 0.008527, Train acc 0.998000, Time 277.4 sec
Epoch 2. Loss: 0.010, Train acc 1.00, Test acc 0.84, Test loss 0.498, Time 334.2 sec, learning_rate 0.000050
Data 16000. Loss: 0.009712, Train acc 0.998062, Time 277.0 sec
Epoch 3. Loss: 0.011, Train acc 1.00, Test acc 0.86, Test loss 0.453, Time 333.6 sec, learning_rate 0.000050
Data 16000. Loss: 0.010765, Train acc 0.997500, Time 277.3 sec
Epoch 4. Loss: 0.010, Train acc 1.00, Test acc 0.85, Test loss 0.463, Time 334.6 sec, learning_rate 0.000050
Data 16000. Loss: 0.008779, Train acc 0.997250, Time 278.2 sec
Epoch 5. Loss: 0.008, Train acc 1.00, Test acc 0.

## save model

In [10]:
net.collect_params().save(save_path + 'method2_' + '02.params')

## predict

In [11]:
def _get_batch(batch, ctx):
    """return data and label on ctx"""
    if isinstance(batch, mx.io.DataBatch):
        data = batch.data[0]
        label = batch.label[0]
    else:
        data, lable = batch
    return (gluon.utils.split_and_load(data, ctx), 
           gluon.utils.split_and_load(label, ctx), 
           data.shape[0])

def net_predict(net, data_iter, ctx = [mx.cpu()]):
    outputs = []
    for i, batch in enumerate(data_iter):
        real_size = batch_size - batch.pad
        data, _, _ = _get_batch(batch, ctx)
        output = [net(x) for x in data]
        outputs.append(nd.concatenate([x.as_in_context(mx.cpu()) for x in output])[0:real_size, :])
        nd.waitall()
        
    outputs = nd.concatenate(outputs)
    return(outputs)


In [12]:
(_, _, test_iter) = get_iter(kv, resize = resize, data_shape = data_shape)

In [13]:
output = net_predict(net, test_iter, ctx = ctx)

In [14]:
output = nd.softmax(output).asnumpy()

## match submission

In [15]:
predict_save_path = project_path + 'predict/'
import pandas as pd
df_pred = pd.read_csv(project_path + 'sample_submission.csv')
test_pictures = pd.read_table(test_path_list, header = None)[2]
df_pred['img'] = test_pictures
for i, j in enumerate(df_pred.columns[1:]):
    df_pred[j] = output[:, i]

df_pred = df_pred.sort_values('img')
df_pred.to_csv(predict_save_path + 'pred6.csv', index = None)